In [4]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

import pandas as pd

In [2]:
patch_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_patches/'

In [ ]:
for stack in ['MD589']:
# for stack in ['MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:

    print stack

    box_x, box_y, box_w, box_h = detect_bbox_lookup[stack]
    first, last = detect_bbox_range_lookup[stack]

    patch_size = 224
    half_size = patch_size/2
    stride = 56

    dm = DataManager(stack=stack)

    ys, xs = np.meshgrid(np.arange(half_size, dm.image_height - half_size, stride), 
                         np.arange(half_size, dm.image_width - half_size, stride),
                         indexing='xy')

    sample_locations = np.c_[xs.flat, ys.flat]


    xmin = max(half_size, box_x*32)
    xmax = min(dm.image_width-half_size-1, (box_x+box_w)*32)
    ymin = max(half_size, box_y*32)
    ymax = min(dm.image_height-half_size-1, (box_y+box_h)*32)

    ######################

    indices_roi_allSections = defaultdict(dict)

    for sec in range(first, last+1):

        dm.set_slice(sec)

        mask = dm.load_thumbnail_mask()
        indices_fg = np.where(mask[sample_locations[:,1]/32, sample_locations[:,0]/32])[0]
        indices_bg = np.setdiff1d(range(sample_locations.shape[0]), indices_fg)

        indices_roi = np.where(np.all(np.c_[sample_locations[:,0] > xmin, sample_locations[:,1] > ymin, 
                                            sample_locations[:,0] < xmax, sample_locations[:,1] < ymax], axis=1))[0]

        indices_roi = np.setdiff1d(indices_roi, indices_bg)
        print len(indices_roi), 'patches in ROI'

        indices_roi_allSections[sec]['roi1'] = indices_roi


    indices_roi_allSections_df = pd.DataFrame(indices_roi_allSections)

    
    table_filepath = os.path.join(patch_dir, '%(stack)s_indices_allROIs_allSections.h5'%{'stack':stack})
    indices_roi_allSections_df.to_hdf(table_filepath, 'indices_allROIs_allSections')
    pd.Series([patch_size, stride, dm.image_width, dm.image_height]).to_hdf(table_filepath, 'grid_parameters')

#     dump_path = os.path.join(patch_dir, '%(stack)s_indices_allROIs_allSections.h5'%{'stack':stack})
#     indices_roi_allSections_df.to_hdf(dump_path, 'indices_allROIs_allSections')
#     pd.Series([patch_size, stride, dm.image_width, dm.image_height]).to_hdf(dump_path, 'grid_parameters')

In [15]:
indices_roi_allSections_df

,150,151,152,153,154,155,156,157,158,159,...,307,308,309,310,311,312,313,314,315,316
roi1,"[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...",...,"[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420..."


In [16]:
# sync with workstation

workstation_patch_dir = '/home/yuncong/CSHL_data_patches'
cmd = 'scp %(fn)s yuncong@132.239.73.151:%(dest)s' % {'fn': table_filepath, 'dest': workstation_patch_dir}
os.system(cmd)

0